In [2]:
import os
import numpy as np
import shutil

# # Creating Train / Val / Test folders (One time use)
root_dir = 'HW8_Data'
cats = '/cats'
dogs = '/dogs'

os.makedirs(root_dir +'/train' + cats)
os.makedirs(root_dir +'/train' + dogs)
os.makedirs(root_dir +'/val' + cats)
os.makedirs(root_dir +'/val' + dogs)

In [32]:
cat_filenames = []
for i in np.arange(12500):
    filename = 'HW8_Data/cat.%d.jpg' % i
    cat_filenames.append(filename)

cat_train = cat_filenames[:10000]
cat_val = cat_filenames[10000:]

In [33]:
dog_filenames = []
for i in np.arange(12500):
    filename = 'HW8_Data/dog.%d.jpg' % i
    dog_filenames.append(filename)

dog_train = dog_filenames[:10000]
dog_val = dog_filenames[10000:]

In [34]:
# Copy-pasting images
for name in cat_train:
    shutil.copy(name, "HW8_Data/train/cats")

for name in cat_val:
    shutil.copy(name, "HW8_Data/validation/cats")

for name in dog_train:
    shutil.copy(name, "HW8_Data/train/dogs")
    
for name in dog_val:
    shutil.copy(name, "HW8_Data/validation/dogs")

In [42]:
import tensorflow as ts
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import SGD

In [102]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

optimizer = SGD(lr=0.002, momentum=0.8)
loss = keras.losses.BinaryCrossentropy()

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy']
)

In [90]:

##################################################    
inputs = keras.Input(shape=(150, 150, 3))

base = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150,150,3))(inputs)

vectors = keras.layers.MaxPooling2D(pool_size=(2,2))(base)

vectors = keras.layers.Flatten()(vectors)

inner = keras.layers.Dense(64, activation='relu')(vectors)

outputs = keras.layers.Dense(1, activation='sigmoid')(inner)

model = keras.Model(inputs, outputs)
##################################################    
optimizer = SGD(lr=0.002, momentum=0.8)
loss = keras.losses.BinaryCrossentropy()

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy', 'std']
)

In [100]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 175232)            0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                11214912  
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 65        
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
_________________________________________________________________


In [74]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [75]:
train_generator = ImageDataGenerator(rescale=1./255)
validation_generator = ImageDataGenerator(rescale=1./255)

In [94]:
train_ds = train_generator.flow_from_directory('./HW8_Data/train/', 
                              target_size=(150,150), 
                              batch_size=20, class_mode='binary')

val_ds = validation_generator.flow_from_directory('./HW8_Data/validation/', 
                              target_size=(150,150), 
                              batch_size=20,
                                    shuffle=False, class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [95]:
train_ds.class_mode

'binary'

In [103]:
history = model.fit(
    train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_ds,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 21s 205ms/step - loss: 0.7244 - accuracy: 0.4816 - val_loss: 0.6781 - val_accuracy: 0.9960
Epoch 2/10
100/100 [==============================] - 21s 213ms/step - loss: 0.6928 - accuracy: 0.5084 - val_loss: 0.6740 - val_accuracy: 0.6480
Epoch 3/10
100/100 [==============================] - 22s 219ms/step - loss: 0.6904 - accuracy: 0.5366 - val_loss: 0.6597 - val_accuracy: 0.5670
Epoch 4/10
100/100 [==============================] - 21s 205ms/step - loss: 0.6894 - accuracy: 0.5548 - val_loss: 0.6933 - val_accuracy: 0.1950
Epoch 5/10
100/100 [==============================] - 21s 215ms/step - loss: 0.6867 - accuracy: 0.5457 - val_loss: 0.6643 - val_accuracy: 0.3890
Epoch 6/10
100/100 [==============================] - 22s 224ms/step - loss: 0.6859 - accuracy: 0.5439 - val_loss: 0.6510 - val_accuracy: 0.4620
Epoch 7/10
100/100 [==============================] - 21s 209ms/step - loss: 0.6871 - accuracy: 0.5357 - val_loss: 0.5799 - val_ac

In [105]:
np.std(history.history['loss'])

0.007482303263297786

In [106]:
train_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_ds = train_generator.flow_from_directory('./HW8_Data/train/', 
                              target_size=(150,150), 
                              batch_size=20, class_mode='binary')

Found 20000 images belonging to 2 classes.


In [107]:
history = model.fit(
    train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_ds,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 26s 262ms/step - loss: 0.6880 - accuracy: 0.5450 - val_loss: 0.7225 - val_accuracy: 0.2010
Epoch 2/10
100/100 [==============================] - 25s 254ms/step - loss: 0.6841 - accuracy: 0.5520 - val_loss: 0.5627 - val_accuracy: 0.8690
Epoch 3/10
100/100 [==============================] - 27s 266ms/step - loss: 0.6856 - accuracy: 0.5525 - val_loss: 0.6424 - val_accuracy: 0.5220
Epoch 4/10
100/100 [==============================] - 27s 269ms/step - loss: 0.6835 - accuracy: 0.5600 - val_loss: 0.6279 - val_accuracy: 0.6230
Epoch 5/10
100/100 [==============================] - 25s 246ms/step - loss: 0.6838 - accuracy: 0.5490 - val_loss: 0.6471 - val_accuracy: 0.5150
Epoch 6/10
100/100 [==============================] - 27s 270ms/step - loss: 0.6722 - accuracy: 0.5885 - val_loss: 0.6055 - val_accuracy: 0.5590
Epoch 7/10
100/100 [==============================] - 28s 280ms/step - loss: 0.6794 - accuracy: 0.5695 - val_loss: 0.6629 - val_ac

In [108]:
np.mean(history.history['val_loss'])

0.6483260333538056

In [109]:
np.mean(history.history['val_accuracy'][5:])

0.4854000061750412